In [1]:
### THIS MAY NOT BE NEEDED FOR DEPLOY SINCE FILES ARE ON GITHUB
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install emoji

In [3]:
import numpy as np
import pandas as pd
import torch
import pickle
import csv
import plotly.graph_objects as go
import re
from emoji import demojize
import string
from string import punctuation
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Loading Trained weights and vocabulary

In [4]:
### CHANGE TO GITHUB REPO WHERE STREAMLIT FILES ARE 
datapath = '/content/drive/MyDrive/Omdena-India-Socio-Economy/'
fmodel = datapath + 'LSTM_RNN_Sentiment_model.pt'
fvocab = datapath + 'LSTM_RNN_Sentiment_vocab.pkl'

In [5]:
print(fmodel)

/content/drive/MyDrive/Omdena-India-Socio-Economy/LSTM_RNN_Sentiment_model.pt


In [6]:
### Load Trained Vocabulary
def load_vocab(fpath):
  pickled_data = open(fpath, 'rb')
  vocab = pickle.load(pickled_data)
  return vocab

vocab_to_int = load_vocab(fvocab)

### Sentiment Model

In [7]:
### SENTIMENT MODEL 
import torch.nn as nn

class SentimentRNN(nn.Module):
    """
    The RNN model that will be used to perform Sentiment analysis.
    """

    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, drop_prob=0.5):
        """
        Initialize the model by setting up the layers.
        """
        super(SentimentRNN, self).__init__()

        self.output_size = output_size
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        
        # embedding and LSTM layers
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(0.3)
        
        # linear and sigmoid layers
        self.fc = nn.Linear(hidden_dim, output_size)
        self.sig = nn.Sigmoid()
        

    def forward(self, x, hidden):
        """
        Perform a forward pass of our model on some input and hidden state.
        """
        batch_size = x.size(0)

        # embeddings and lstm_out
        x = x.long()
        embeds = self.embedding(x)
        lstm_out, hidden = self.lstm(embeds, hidden)
    
        # stack up lstm outputs
        lstm_out = lstm_out.contiguous().view(-1, self.hidden_dim)
        
        # dropout and fully-connected layer
        out = self.dropout(lstm_out)
        out = self.fc(out)
        # sigmoid function
        sig_out = self.sig(out)
        
        # reshape to be batch_size first
        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get last batch of labels
        
        # return last sigmoid output and hidden state
        return sig_out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x hidden_dim,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        
        return hidden
        

### Functions to pre-process input text

In [8]:
def clean_text(text):
    # Convert Emoji to strings
    #text = demojize(text)

    # Remove HTML Tags
    try:
        text = html.document_fromstring(text).text_content()
    except:
        pass
          
    # Remove Hyperlinks
    text = re.sub('http\S+', ' ', text)
    
    # Remove html character codes ( like &amp)
    text = re.sub('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});', ' ', text)
    
    # Remove Hashtags
    text = re.sub("#([a-zA-Z0-9_]{1,50})", ' ', text)
    
    # Remove Mentions
    text = re.sub("@([a-zA-Z0-9_]{1,50})", ' ', text)

    # Remove non alphabets
    text = re.sub('[^a-zA-Z ]+', ' ', text)    
    
    # Remove punctuations
    text = re.sub('[^-9A-Za-z ]', ' ' , text)

    # Lowercase and split - converts to list
    text = text.lower().split()  
    
    # Remove stop and short words
    text = [word for word in text if word not in stop and len(word) > 1]
    
    # Stemming
    # text = [stemmer.stem(word) for word in text]

    # Join and Return
    return ' '.join(text )

def tokenize_text(text):

    # splitting by spaces
    test_words = text.split()

    # tokens
    test_ints = []
    test_ints.append([vocab_to_int[word] for word in test_words if vocab_to_int.get(word)])

    return test_ints


In [9]:
def pad_features(test_ints, seq_length):
    ''' Return features of review_ints, where each review is padded with 0's 
        or truncated to the input seq_length.
    '''
    
    # getting the correct rows x cols shape
    features = np.zeros((len(test_ints), seq_length), dtype=int)

    # for each review, I grab that review and 
    for i, row in enumerate(test_ints):
      if len(row) > 0:   # If word is not in vocabulary then there will no integer representation    
        features[i, -len(row):] = np.array(row)[:seq_length]
    
    return features

### Instantiating Model

In [10]:
# Instantiate the model w/ hyperparams
vocab_size = len(vocab_to_int)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_dim = 400
hidden_dim = 256
n_layers = 2

net = SentimentRNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers)

print(net)

SentimentRNN(
  (embedding): Embedding(60399, 400)
  (lstm): LSTM(400, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (sig): Sigmoid()
)


### Load pre-trained weights

In [11]:
# Load Trained Model weights
model = net.load_state_dict(torch.load(fmodel, map_location=torch.device('cpu')))
print(model)

<All keys matched successfully>


### Call Function

In [12]:
def predict(net, test_text, sequence_length=200):
    
    net.eval()
    
    # tokenize text
    text = clean_text(test_text)
    test_ints = tokenize_text(text)
    
    # pad tokenized sequence
    seq_length=sequence_length
    features = pad_features(test_ints, seq_length)
    
    # convert to tensor to pass into your model
    feature_tensor = torch.from_numpy(features)
    
    batch_size = feature_tensor.size(0)
    
    # initialize hidden state
    h = net.init_hidden(batch_size)
  
    
    # get the output from the model
    output, h = net(feature_tensor, h)
    
    # convert output probabilities to predicted class (0 or 1)
    pred = torch.round(output.squeeze()) 
    response = int(pred.item())
    pred_value = output.item()

    #if(pred.item()==1):
    #    response = "Positive sentiment"
    #else:
    #    response = "Negative sentiment"
    
    

    return text, response, pred_value


## Example Input and Output

In [13]:
input_text = "people happy"
seq_length=51 # good to use the length that was trained on
text, response, pred_value = predict(net, input_text, seq_length)
print(text)
print(response)
print('Prediction value, pre-rounding: {:.6f}'.format(pred_value))

people happy
1
Prediction value, pre-rounding: 0.998327


In [14]:
input_text = "covid was devastating"
seq_length=51 # good to use the length that was trained on
text, response, pred_value = predict(net, input_text, seq_length)
print(text)
print(response)
print('Prediction value, pre-rounding: {:.6f}'.format(pred_value))

covid devastating
0
Prediction value, pre-rounding: 0.000275


## Processing Real time tweet files

In [15]:
### THIS FILE WILL CHANGE DAILY TO REFLECT REAL TIME I.E. DAILY TWEETS
fdate = datapath + "July_31_2021.txt"

In [16]:
def process_file(fdate):
  data = pd.read_csv(fdate, delimiter = "\t", quoting=csv.QUOTE_NONE, header=None)
  data.rename({0:'text'}, axis=1, inplace=True)
  a = list(data['text'])
  sentiment = []
  for input_text in a:
    if len(input_text.split(' ')) > 10:
      text, response, pred_value = predict(net, input_text, seq_length)
      sentiment.append(response)
  return sentiment

In [17]:
s = process_file(fdate)

In [18]:
print("Number of Tweets Processed", len(s))

Number of Tweets Processed 317


In [19]:
from collections import Counter
c = Counter(s)

In [20]:
x = ['Positive Sentiment', 'Negative Sentiment']
y = [c.get(1), c.get(0)]

In [21]:
sum(y)

317

In [22]:
# Specify the plots
bar_plots = [
             go.Bar(x=x, y=y, name='Sentiment', marker=go.bar.Marker(color='#0343df'))
             ]


In [23]:
layout = go.Layout(
title=go.layout.Title(text="Sentiment Analysis", x=0.5),
yaxis_title="Count",
xaxis_tickmode="array",
xaxis_tickvals=list(range(27)),
xaxis_ticktext=tuple(x),
font=dict(
        family="Courier New, monospace",
        size=24,
        color="Black"
    )
)

In [24]:
# Make the multi-bar plot
fig = go.Figure(data=bar_plots, layout=layout)
fig.show()